# Lab 2: Deep learning in Keras

Na druhom cvičení si vytvoríme našu prvú hlbokú neurónovú sieť pomocou knižnice Keras. Pred tým sa však oboznámime s ďalšími knižnicami, ktoré sa bežne používajú v Pythone pri spracovaní a analýze údajov. Budeme používať každého obľúbený Iris dataset.

**Zdroje**

[Iris dataset](https://github.com/ianmagyar/dl-course/blob/master/labs/sources/iris.csv)

[Dokumentácia Keras - Modely](https://keras.io/models/sequential/)

[Dokumentácia Keras - Vrstvy](https://keras.io/layers/core/)

## 1. Načítanie datasetu - Pandas

Síce v Pythone máme rôzne možnosti pre načítanie datasetov a pre prácu s nimi, veľmi často sa používa knižnica Pandas. Pandas je knižnica pre analýzu údajov s vysokým výkonom a veľkou mierou optimalizácie. Naimportujeme ju už známym spôsobom.

In [ ]:
import pandas as pd

Pandas nám umožňuje načítať dataset priamo zo súboru [CSV](https://en.wikipedia.org/wiki/Comma-separated_values), a očakáva že prvý riadok obsahuje názvy jednotlivých stĺpcov. Knižnica ďalej podporuje typy súborov ako HTML, JSON, HDF5, SQL a ďalšie.

In [ ]:
# TODO: read dataset from csv file (add path to file)
dataset = pd.read_csv('# TODO')
print(dataset)

Načítaný dataset má typ DataFrame. K ľubovoľným stĺpcom sa dostaneme zadaním názvu stĺpca ako index datasetu. Ak chceme zobraziť viac stĺpcov, index musí byť zoznam s názvami týchto stĺpcov.

In [ ]:
# TODO: select only column SepalLengthCm
print(dataset['# TODO'])

# TODO: select columns SepalLengthCm and SepalWidthCm
print(dataset['# TODO'])

Alternatívne vieme zobraziť stĺpce ako keby boli parametrom objektu dataset, alebo vieme použiť aj poradové číslo stĺpca (znak `:` pred čiarkou vyjadruje všetky riadky).

In [ ]:
print(dataset.SepalLengthCm)
print(dataset.iloc[:, 0])

K riadkom prístupujeme cez číselné indexy, pričom dokopy ich môžeme mať až tri. Prvé číslo vyjadruje poradové číslo prvého riadku, druhé číslo poradové číslo posledného riadku (vľavo uzavretý interval, podľa [pravidiel indexovania v Pythone](https://www.digitalocean.com/community/tutorials/how-to-index-and-slice-strings-in-python-3)), a tretie číslo step. Takto vieme napríklad vypísať každý druhý riadok z intervalu 1-10:

In [ ]:
print(dataset[0:10:2])

Alternatívne môžete použiť aj `loc` funkciu DataFrame-ov (podľa index atribútu; druhý index vyjadruje uzavretý interval), alebo `iloc` funkciu (podľa poradia; druhý index vyjadruje uzavretý interval)

In [ ]:
print(dataset.loc[0:9:2])
print(dataset.iloc[0:9:2])

Indexovanie riadkov a stĺpcov viete aj kombinovať, na poradí nezáleží:

In [ ]:
print(dataset[:10:2]['SepalLengthCm'])
print(dataset['SepalLengthCm'][:10:2])

Z datasetu viete vybrať iba niektoré riadky aj na základe hodnoty niektorého atribútu použitím `lambda` funkcie. Napríklad, pre všetky riadky, kde hodnota SepalLengthCm je viac ako 5:

In [ ]:
print(dataset.loc[lambda df:df.SepalLengthCm > 5, :])

Všetky tieto podmnožiny majú typ `DataFrame`. Ak chcete ich použiť ako zoznam, resp. zoznam zoznamov, musíte pridať `values`:

In [ ]:
dataset['SepalLengthCm'][:10:].values

## 2. Vizualizácia údajov - Seaborn

Pred tým, než zadefinujeme našu sieť, potrebujeme získať intuitívne pochopenie datasetu a vzťahov medzi jednotlivými atribútmi a výsledkom klasifikácie. V tomto nám pomôže knižnica Seaborn, ktorá slúži na vizualizáciu údajov a využíva to knižnicu matplotlib.

In [ ]:
# import all needed libraries
import seaborn as sns
import matplotlib.pyplot as plt

# set plot style
sns.set(style="ticks")
sns.set_palette("husl")

# create plots over all dataset; for subset use iloc indexing
sns.pairplot(dataset, hue="Species")

# display plots using matplotlib
plt.show()

## 3. Príprava datasetu

Pre trénovanie neurónovej siete potrebujeme vytvoriť trénovaciu a testovaciu množinu. Najprv však musíme rozdeliť vstupy siete (atribúty) od výstupu (výsledok klasifikácie).

In [ ]:
# split data into input (X - select the first four columns) and output (y - select last column)
X = dataset.iloc[# TODO].values
y = dataset.iloc[# TODO].values

Pri výstupe však nastáva problém: kým neurónová sieť má na výstupe číselné hodnoty, náš dataset stále obsahuje reťazce ako deskriptory tried. Preto potrebujeme tieto reťazce premeniť na vektorovú reprezentáciu (Iris-setosa -> (1 0 0), atď.). V tomto nám pomôže `LabelEncoder` z knižnice `scikit-learn`.

In [ ]:
from sklearn.preprocessing import LabelEncoder

encoder = LabelEncoder()
# transform string labels into number values 0, 1, 2
y1 = encoder.fit_transform(y)

# transform number values into vector representation
Y = pd.get_dummies(y1).values

Ďalšou úlohou je rozdelenie množiny na trénovaciu a testovaciu. Na to použijeme ďalšiu funkciu z knižnice `scikit-learn`, a to `train_test_split` ([dokumentácia](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html)), ktorá zachová poradie vstupov a výstupov a má tri dôležité parametre:
1. zoznam vstupov
2. zoznam výstupov
3. test_size - veľkosť testovacej množiny medzi 0 a 1 (môžete použiť aj train_size)

Pre opakovateľnosť trénovania je odporúčané používať random seed zadaním parametra `random_state`.

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(# TODO)

## 4. Definícia a trénovanie neurónovej siete - Keras

Teraz nám už naozaj nič nebráni definovať našu neurónovú sieť pomocou knižnice Keras. Pre definíciu jednoduchej siete v Kerase potrebujeme tri veci:
1. model - v tomto kroku použijeme jednoduchý feed-forward sekvenčný model ([dokumentácia](https://keras.io/models/sequential/))
2. vrstvy - použijeme iba fully connected dense vrstvy ([dokumentácia](https://keras.io/layers/core/#dense))
3. optimizátor - algoritmus, ktorý nám zadefinuje spôsob trénovania siete; my použijeme optimizátor Adam ([dokumentácia](https://keras.io/optimizers/#adam))

Najprv naimportujeme potrebné triedy:

In [ ]:
from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import Adam

Následne vytvoríme model a pridáme doňho vrstvy. Pri definícii vrstiev potrebujeme zadať počet neurónov vo vrstve a aktivačnú funkciu. Špecifická je prvá vrstva v sieti, keďže pre ňu potrebujeme zadefinovať aj počet vstupných neurónov. Počet neurónov v poslednej vrstve má zodpovedať formátu výstupu siete.

In [ ]:
model = Sequential()

model.add(Dense(10,input_shape=(4,),activation='tanh'))
# TODO: add dense layer with 8 neurons and tanh activation function
model.add()
# TODO: add dense layer with 6 neurons and tanh activation function
model.add())
# TODO: add dense layer with 3 neurons and softmax activation function
model.add()

Pred trénovaním siete ešte potrebujeme zadefinovať spôsob trénovania cez optimizátor. Urobíme tak zavolaním funkcie `compile` ([dokumentácia](https://keras.io/models/sequential/#compile)) s nasledujúcimi parametrami:
* optimizer (optimizátor)
* loss function (chybová funkcia)
* metrics (metriky) - nie je povinné, slúži iba na vyhodnotenie neurónovej siete

In [ ]:
# TODO: add Adam optimizer with a learning rate of 0.04
# TODO: add loss function categorical_crossentropy
model.compile(# TODO, # TODO,metrics=['accuracy'])

Základné informácie o siete ako aj jej topológiu získame pomocou funkcie `summary`.

In [ ]:
model.summary()

Ak sme spokojní so sieťou, môžeme začať ju trénovať pomocou fukcie `fit` ([dokumentácia](https://keras.io/models/sequential/#fit)). V tomto kroku použijeme iba niektoré parametre funkcie:
* vstupy - pole s vstupmi z trénovacej množiny
* výstupy - pole so správnymi výstupmi z trénovacej množiny
* počet epoch - epocha je jedna iterácia nad celou trénovacou množinou

Ďalšie často používané parametre:
* batch_size - batch slúži na presnejšiu gradient aktualizáciu
* verbose - definuje, či chceme zobraziť progress počas trénovania

In [ ]:
model.fit(X_train, y_train, epochs=100)

Teraz nám neostáva nič iné len vyhodnotiť našu neurónovú sieť. Najprv musíme získať výstupy, ktoré nám generuje neurónová sieť pre vstupy z testovacej množiny ([dokumentácia](https://keras.io/models/sequential/#predict)).

In [ ]:
y_pred = model.predict(X_test)

Ďalej porovnáme ozajstné výstupy s očakávanými. Keďže výstup má vektorovú reprezentáciu, potrebujeme zistiť pozíciu kde sa nachádza najväčšia hodnota vo vektore. V tomto nám pomôže knižnica `numpy`, ktorú sme zatiaľ nepoužili explicitne, ale podporuje všetky už použité knižnice. Jedná sa o efektívne a optimalizované riešenie práce s poľami.

Pre vyhodnotenie našej siete použijeme konfúznu maticu.

In [ ]:
import numpy as np

y_test_class = np.argmax(y_test,axis=1)
y_pred_class = np.argmax(y_pred,axis=1)

from sklearn.metrics import classification_report, confusion_matrix

print(classification_report(y_test_class, y_pred_class))
print(confusion_matrix(y_test_class, y_pred_class))